# Settings

In [1]:
# Boilerplate settings

import arcpy

arcpy.env.overwriteOutput = True

storagefolder = r"C:\Users\16316\Documents\ArcGIS\Packages\ShrinkingChallenge_0bd0fc\p20\shrinkingchallenge.gdb"

arcpy.env.workspace = storagefolder

In [2]:
# Pull up feature class

fc = storagefolder + r'\numbers'

# Keep polygons separate method

In [4]:
# Create a list to hold a copy of the coordinates for the three polygons
# Within this list
# (1) Each sub list will be a polygon
# (2) Each sub sub list a coordinate pair in the polygon 

fc_copy = []


# Use the search cursor to access the polygon's geometery 

with arcpy.da.SearchCursor(fc,['Shape@']) as rows:
    
    # For each polygon
        
    for row in rows:
                
        for part in row[0]:
            
            # Make a sub list for every polygon
            
            sub_lst = []
            for pnt in part:
                
                # Make a sub list for every coordinate pair
                
                sub_lst_coord = []
                
                if pnt:
                    sub_lst_coord.append(pnt.X)
                    sub_lst_coord.append(pnt.Y)
                
                # Add each coordinate pair list to the polygon sublist 
                
                sub_lst.append(sub_lst_coord)
                
                del sub_lst_coord

            # Add each polygon list to the fc_copy list
            
            fc_copy.append(sub_lst)
            del sub_lst



# Create a list to hold the ultimate feature class
# This list will contain three polygon items

fc_shrunk_lst = []           

# For each polygon list in features
# Make an array
# Cycle through the coordinates and divide them by two
# Add them to the array
# Make a polygon out of that array
# Add that polygon to the fc_list. This will ultimately be the feature class

for polygon_lst in fc_copy:
    array_of_points = arcpy.Array()
    for coord in polygon_lst:
        single_point = arcpy.Point()
        single_point.X = coord[0]/(2.0)
        single_point.Y = coord[1]/(2.0)
        array_of_points.add(single_point)
        del single_point
    new_poly = arcpy.Polygon(array_of_points)
    fc_shrunk_lst.append(new_poly)


 # Make new feature class from fc list 

arcpy.CopyFeatures_management(fc_shrunk_lst, "numbers_shrunk_separate")

<Result 'C:\\Users\\16316\\Documents\\ArcGIS\\Packages\\ShrinkingChallenge_0bd0fc\\p20\\shrinkingchallenge.gdb\\numbers_shrunk_separate'>

# Make single multipart polygon method

In [5]:
# With much help from https://community.esri.com/thread/191831-need-help-creating-multipart-polygon-output-using-arcpy

# Create a list to hold the ultimate multipart poylgon 

features = []

# Use the search cursor to access the polygon's geometery 

with arcpy.da.SearchCursor(fc,['Shape@']) as rows:
    
    # For each polygon, create an array to hold that polygon's points
    
    array_of_points = arcpy.Array()
    
    for row in rows:
        
        # for each polygon, cycle through the points and add them to a subarray. 
        # This subarray represents one polygon in the the multipart polygons
        
        for part in row[0]:
            sub_array = arcpy.Array()
            for pnt in part:
                if pnt:
                    single_point = arcpy.Point()
                    single_point.X = (pnt.X/2.0)
                    single_point.Y = (pnt.Y/2.0)
                    sub_array.add(single_point)            
            
            sub_array.add(sub_array.getObject(0))
            
            # Add each polygon's subarray to a master array - the master polygon. 
            
            array_of_points.add(sub_array)
        
            del sub_array
    
    # Create a new polygon based on the array of points
    # Add it to the list 
    
    
    new_poly = arcpy.Polygon(array_of_points)
    features.append(new_poly)
    del array_of_points
    del new_poly

    
# Make new feature class based on the list of polygons
    
arcpy.CopyFeatures_management(features, "numbers_shrunk_multipart")

<Result 'C:\\Users\\16316\\Documents\\ArcGIS\\Packages\\ShrinkingChallenge_0bd0fc\\p20\\shrinkingchallenge.gdb\\numbers_shrunk_multipart'>